In [1]:
# 간단한 Manus 스타일 에이전트 (OpenAI GPT 기반)
# 주요 기능: 유저 명령 해석, 코드 실행, 파일 생성 등

import openai
import os
import subprocess

openai.api_key = os.getenv("OPENAI_API_KEY")  # 환경변수로 API 키 설정

def ask_agent(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful autonomous AI agent that completes tasks as instructed."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

def handle_task(task):
    print(f"[사용자 지시] {task}")
    
    # 1. 지시를 GPT에 전달하여 계획 수립
    plan_prompt = f"""
    사용자가 다음과 같은 작업을 요청했습니다:
    "{task}"

    이 작업을 완료하기 위한 단계별 계획을 짧고 명확하게 작성하세요.
    """
    plan = ask_agent(plan_prompt)
    print("[계획 수립]")
    print(plan)

    # 2. 코드가 필요한 경우 생성 요청
    if "코드" in task or "스크립트" in task or "자동화" in task:
        code_prompt = f"""
        다음 작업을 자동화하기 위한 파이썬 코드를 작성하세요:
        {task}
        코드만 출력하세요.
        """
        code = ask_agent(code_prompt)
        with open("task_script.py", "w") as f:
            f.write(code)
        print("[코드 생성 완료 → task_script.py]")
        
        try:
            result = subprocess.check_output(["python", "task_script.py"], stderr=subprocess.STDOUT, text=True)
            print("[실행 결과]")
            print(result)
        except subprocess.CalledProcessError as e:
            print("[오류 발생]")
            print(e.output)
    else:
        # 단순 정보형 작업이면 응답 출력
        print("[응답]")
        print(ask_agent(task))

if __name__ == "__main__":
    while True:
        task = input("명령어 입력 > ")
        if task in ["exit", "quit"]:
            break
        handle_task(task)


ModuleNotFoundError: No module named 'openai'